In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get the coffee sales data
df = pd.read_csv("index.csv")

# Convert date and datetime columns to proper datetime format
df['date'] = pd.to_datetime(df['date'])
df['datetime'] = pd.to_datetime(df['datetime'])

# Summary statistics
summary_stats = df.describe(include='all')

# Sales by date
daily_sales = df.groupby('date').agg({
    'money': ['sum', 'count', 'mean'],
    'coffee_name': 'count'
}).reset_index()
daily_sales.columns = ['date', 'total_revenue', 'transaction_count', 'avg_transaction', 'coffee_count']

# Sales by coffee type
coffee_sales = df.groupby('coffee_name').agg({
    'money': ['sum', 'count', 'mean'],
    'date': 'nunique'
}).reset_index()
coffee_sales.columns = ['coffee_name', 'total_revenue', 'total_sales', 'avg_price', 'days_sold']
coffee_sales = coffee_sales.sort_values('total_sales', ascending=False)

# Payment method analysis
payment_analysis = df.groupby('cash_type').agg({
    'money': ['sum', 'count', 'mean'],
    'coffee_name': 'count'
}).reset_index()
payment_analysis.columns = ['payment_method', 'total_revenue', 'transaction_count', 'avg_transaction', 'coffee_count']

# Monthly sales
df['month'] = df['date'].dt.to_period('M')
monthly_sales = df.groupby('month').agg({
    'money': ['sum', 'count'],
    'coffee_name': 'count'
}).reset_index()
monthly_sales.columns = ['month', 'total_revenue', 'transaction_count', 'coffee_count']
monthly_sales['month'] = monthly_sales['month'].dt.to_timestamp()

# Weekly pattern analysis
df['day_of_week'] = df['date'].dt.day_name()
weekly_sales = df.groupby('day_of_week').agg({
    'money': ['sum', 'count'],
    'coffee_name': 'count'
}).reset_index()
weekly_sales.columns = ['day_of_week', 'total_revenue', 'transaction_count', 'coffee_count']

# Create subplots for comprehensive visualization
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Daily Sales Trend',
        'Coffee Type Popularity',
        'Monthly Revenue Trend',
        'Payment Method Breakdown',
        'Weekly Sales Pattern',
        'Top 10 Coffee Types'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"type": "domain"}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Daily sales trend
fig.add_trace(
    go.Scatter(x=daily_sales['date'], y=daily_sales['total_revenue'],
               name='Daily Revenue', line=dict(color='blue')),
    row=1, col=1
)

# Coffee type popularity (top 10)
top_coffees = coffee_sales.head(10)
fig.add_trace(
    go.Bar(x=top_coffees['coffee_name'], y=top_coffees['total_sales'],
           name='Top Coffee Types', marker_color='orange'),
    row=1, col=2
)

# Monthly revenue trend
fig.add_trace(
    go.Scatter(x=monthly_sales['month'], y=monthly_sales['total_revenue'],
               name='Monthly Revenue', line=dict(color='green')),
    row=2, col=1
)

# Payment method breakdown
fig.add_trace(
    go.Pie(labels=payment_analysis['payment_method'], values=payment_analysis['total_revenue'],
           name='Payment Methods', hole=0.3),
    row=2, col=2
)

# Weekly sales pattern
fig.add_trace(
    go.Bar(x=weekly_sales['day_of_week'], y=weekly_sales['total_revenue'],
           name='Weekly Pattern', marker_color='purple'),
    row=3, col=1
)

# Top 10 coffee types by revenue
top_revenue_coffees = coffee_sales.head(10)
fig.add_trace(
    go.Bar(x=top_revenue_coffees['coffee_name'], y=top_revenue_coffees['total_revenue'],
           name='Top Revenue Coffees', marker_color='red'),
    row=3, col=2
)

# Update layout
fig.update_layout(
    title='Comprehensive Coffee Sales Analysis',
    height=900,
    showlegend=False,
    plot_bgcolor='white'
)

# Update x-axis for weekly pattern to show proper order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fig.update_xaxes(categoryorder='array', categoryarray=day_order, row=3, col=1)

fig.show()

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load data
df = pd.read_csv("index.csv")
df['date'] = pd.to_datetime(df['date'])

# Aggregate daily sales
daily_sales = df.groupby('date').agg({'money':'sum'}).reset_index()
daily_sales.columns = ['date', 'total_revenue']
daily_sales['total_revenue'] = pd.to_numeric(daily_sales['total_revenue'])

# Set daily frequency and fill missing dates
daily_sales.set_index('date', inplace=True)
daily_sales = daily_sales.asfreq('D').fillna(method='ffill')

# ----------------- ARIMA Model -----------------
arima_model = ARIMA(daily_sales['total_revenue'], order=(5,1,0))
arima_results = arima_model.fit()

# Forecast next 30 days
arima_forecast = arima_results.get_forecast(steps=30)
arima_forecast_df = arima_forecast.summary_frame()
arima_forecast_df['date'] = pd.date_range(start=daily_sales.index[-1] + pd.Timedelta(days=1), periods=30)
arima_forecast_df.set_index('date', inplace=True)

# ----------------- Random Forest Model -----------------
# Create features for RF
rf_data = daily_sales.copy()
rf_data['day_of_year'] = rf_data.index.dayofyear
rf_data['week_of_year'] = rf_data.index.isocalendar().week
rf_data['month'] = rf_data.index.month
rf_data['day_of_week'] = rf_data.index.dayofweek

# Lag features
for lag in range(1, 8):
    rf_data[f'lag_{lag}'] = rf_data['total_revenue'].shift(lag)

rf_data.dropna(inplace=True)

# Train Random Forest on all available data
feature_cols = ['day_of_year', 'week_of_year', 'month', 'day_of_week'] + [f'lag_{lag}' for lag in range(1, 8)]
X_train = rf_data[feature_cols]
y_train = rf_data['total_revenue']

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ----------------- Forecast 30 days using RF -----------------
future_dates = pd.date_range(start=daily_sales.index[-1] + pd.Timedelta(days=1), periods=30)
future_preds = []
last_values = daily_sales['total_revenue'][-7:].tolist()

for d in future_dates:
    features = [
        d.dayofyear,
        d.isocalendar().week,
        d.month,
        d.dayofweek
    ] + last_values[-7:]
    pred = rf_model.predict([features])[0]
    future_preds.append(pred)
    last_values.append(pred)

rf_forecast_df = pd.DataFrame({'total_revenue': future_preds}, index=future_dates)

# ----------------- Calculate RMSE (optional, on last 30 days if needed) -----------------
# Here we just show ARIMA and RF future forecast, not RMSE

# ----------------- Plotting -----------------
fig = go.Figure()

# Actual sales
fig.add_trace(go.Scatter(
    x=daily_sales.index, y=daily_sales['total_revenue'],
    name='Actual Daily Sales', line=dict(color='blue')
))

# ARIMA forecast
fig.add_trace(go.Scatter(
    x=arima_forecast_df.index, y=arima_forecast_df['mean'],
    name='ARIMA Forecast (30 days)', line=dict(color='red', dash='dash')
))

# Random Forest forecast
fig.add_trace(go.Scatter(
    x=rf_forecast_df.index, y=rf_forecast_df['total_revenue'],
    name='Random Forest Forecast (30 days)', line=dict(color='green', dash='dot')
))

# Layout
fig.update_layout(
    title='Daily Sales Forecasting - ARIMA vs Random Forest',
    xaxis_title='Date',
    yaxis_title='Total Revenue',
    plot_bgcolor='white'
)

fig.show()

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\237315122.py:19: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoje

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor

# Load data
df = pd.read_csv("index.csv")
df['date'] = pd.to_datetime(df['date'])

# Aggregate weekly sales (Monday as start of week)
weekly_sales = df.groupby(pd.Grouper(key='date', freq='W-MON')).agg({'money':'sum'}).reset_index()
weekly_sales.columns = ['week', 'total_revenue']
weekly_sales['total_revenue'] = pd.to_numeric(weekly_sales['total_revenue'])

# Set weekly frequency and fill missing weeks
weekly_sales.set_index('week', inplace=True)
weekly_sales = weekly_sales.asfreq('W-MON').fillna(method='ffill')

# ----------------- ARIMA Model -----------------
arima_model = ARIMA(weekly_sales['total_revenue'], order=(3,1,0))
arima_results = arima_model.fit()

# Forecast next 12 weeks
arima_forecast = arima_results.get_forecast(steps=12)
arima_forecast_df = arima_forecast.summary_frame()
arima_forecast_df['week'] = pd.date_range(start=weekly_sales.index[-1] + pd.Timedelta(weeks=1), periods=12, freq='W-MON')
arima_forecast_df.set_index('week', inplace=True)

# ----------------- Random Forest Model -----------------
rf_data = weekly_sales.copy()
rf_data['week_of_year'] = rf_data.index.isocalendar().week
rf_data['month'] = rf_data.index.month
rf_data['quarter'] = rf_data.index.quarter

# Lag features
for lag in range(1, 5):
    rf_data[f'lag_{lag}'] = rf_data['total_revenue'].shift(lag)

rf_data.dropna(inplace=True)

# Train on all available data
feature_cols = ['week_of_year', 'month', 'quarter'] + [f'lag_{lag}' for lag in range(1, 5)]
X_train = rf_data[feature_cols]
y_train = rf_data['total_revenue']

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ----------------- Forecast 12 weeks using RF -----------------
future_weeks = pd.date_range(start=weekly_sales.index[-1] + pd.Timedelta(weeks=1), periods=12, freq='W-MON')
future_preds = []
last_values = weekly_sales['total_revenue'][-4:].tolist()  # last 4 weeks for lag features

for w in future_weeks:
    features = [
        w.isocalendar().week,
        w.month,
        w.quarter
    ] + last_values[-4:]
    pred = rf_model.predict([features])[0]
    future_preds.append(pred)
    last_values.append(pred)

rf_forecast_df = pd.DataFrame({'total_revenue': future_preds}, index=future_weeks)

# ----------------- Plotting -----------------
fig = go.Figure()

# Actual weekly sales
fig.add_trace(go.Scatter(
    x=weekly_sales.index, y=weekly_sales['total_revenue'],
    name='Actual Weekly Sales', line=dict(color='blue')
))

# ARIMA forecast
fig.add_trace(go.Scatter(
    x=arima_forecast_df.index, y=arima_forecast_df['mean'],
    name='ARIMA Forecast (12 weeks)', line=dict(color='red', dash='dash')
))

# Random Forest forecast
fig.add_trace(go.Scatter(
    x=rf_forecast_df.index, y=rf_forecast_df['total_revenue'],
    name='Random Forest Forecast (12 weeks)', line=dict(color='green', dash='dot')
))

# Layout
fig.update_layout(
    title='Weekly Sales Forecasting - ARIMA vs Random Forest',
    xaxis_title='Week',
    yaxis_title='Total Revenue',
    plot_bgcolor='white'
)

fig.show()


C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\106083943.py:18: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoje

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor

# Load data
df = pd.read_csv("index.csv")
df['date'] = pd.to_datetime(df['date'])

# Aggregate monthly sales
monthly_sales = df.groupby(pd.Grouper(key='date', freq='M')).agg({'money':'sum'}).reset_index()
monthly_sales.columns = ['month', 'total_revenue']
monthly_sales['total_revenue'] = pd.to_numeric(monthly_sales['total_revenue'])

# Set monthly frequency and fill missing months
monthly_sales.set_index('month', inplace=True)
monthly_sales = monthly_sales.asfreq('M').fillna(method='ffill')

# ----------------- ARIMA Model -----------------
arima_model = ARIMA(monthly_sales['total_revenue'], order=(2,1,0))
arima_results = arima_model.fit()

# Forecast next 6 months
arima_forecast = arima_results.get_forecast(steps=6)
arima_forecast_df = arima_forecast.summary_frame()
arima_forecast_df['month'] = pd.date_range(start=monthly_sales.index[-1] + pd.offsets.MonthEnd(1), periods=6, freq='M')
arima_forecast_df.set_index('month', inplace=True)

# ----------------- Random Forest Model -----------------
rf_data = monthly_sales.copy()
rf_data['month_of_year'] = rf_data.index.month
rf_data['quarter'] = rf_data.index.quarter

# Lag features
for lag in range(1, 4):
    rf_data[f'lag_{lag}'] = rf_data['total_revenue'].shift(lag)

rf_data.dropna(inplace=True)

# Train on all available data
feature_cols = ['month_of_year', 'quarter'] + [f'lag_{lag}' for lag in range(1, 4)]
X_train = rf_data[feature_cols]
y_train = rf_data['total_revenue']

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ----------------- Forecast 6 months using RF -----------------
future_months = pd.date_range(start=monthly_sales.index[-1] + pd.offsets.MonthEnd(1), periods=6, freq='M')
future_preds = []
last_values = monthly_sales['total_revenue'][-3:].tolist()  # last 3 months for lag features

for m in future_months:
    features = [
        m.month,
        m.quarter
    ] + last_values[-3:]
    pred = rf_model.predict([features])[0]
    future_preds.append(pred)
    last_values.append(pred)

rf_forecast_df = pd.DataFrame({'total_revenue': future_preds}, index=future_months)

# ----------------- Plotting -----------------
fig = go.Figure()

# Actual monthly sales
fig.add_trace(go.Scatter(
    x=monthly_sales.index, y=monthly_sales['total_revenue'],
    name='Actual Monthly Sales', line=dict(color='blue')
))

# ARIMA forecast
fig.add_trace(go.Scatter(
    x=arima_forecast_df.index, y=arima_forecast_df['mean'],
    name='ARIMA Forecast (6 months)', line=dict(color='red', dash='dash')
))

# Random Forest forecast
fig.add_trace(go.Scatter(
    x=rf_forecast_df.index, y=rf_forecast_df['total_revenue'],
    name='Random Forest Forecast (6 months)', line=dict(color='green', dash='dot')
))

# Layout
fig.update_layout(
    title='Monthly Sales Forecasting - ARIMA vs Random Forest',
    xaxis_title='Month',
    yaxis_title='Total Revenue',
    plot_bgcolor='white'
)

fig.show()

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\72694148.py:12: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\72694148.py:18: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\72694148.py:18: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\72694148.py:27: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\72694148.py:50: FutureWarning:

'M' is deprecated and will be removed in a f

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor

# Load data
df = pd.read_csv("index.csv")
df['date'] = pd.to_datetime(df['date'])

# Create subplots
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        'Daily Sales Forecasting',
        'Weekly Sales Forecasting',
        'Monthly Sales Forecasting'
    ),
    vertical_spacing=0.1
)

# -------------------- DAILY FORECASTING --------------------
daily_sales = df.groupby('date')['money'].sum().reset_index()
daily_sales.columns = ['date', 'total_revenue']
daily_sales.set_index('date', inplace=True)
daily_sales = daily_sales.asfreq('D').fillna(method='ffill')

# ARIMA
arima_daily = ARIMA(daily_sales['total_revenue'], order=(5,1,0)).fit()
arima_forecast_daily = arima_daily.get_forecast(steps=30).summary_frame()
arima_forecast_daily['date'] = pd.date_range(daily_sales.index[-1] + pd.Timedelta(days=1), periods=30)
arima_forecast_daily.set_index('date', inplace=True)

# Random Forest
rf_daily = daily_sales.copy()
rf_daily['day_of_year'] = rf_daily.index.dayofyear
rf_daily['week_of_year'] = rf_daily.index.isocalendar().week
rf_daily['month'] = rf_daily.index.month
rf_daily['day_of_week'] = rf_daily.index.dayofweek
for lag in range(1, 8):
    rf_daily[f'lag_{lag}'] = rf_daily['total_revenue'].shift(lag)
rf_daily.dropna(inplace=True)
feature_cols_daily = ['day_of_year','week_of_year','month','day_of_week'] + [f'lag_{lag}' for lag in range(1,8)]
rf_model_daily = RandomForestRegressor(n_estimators=100, random_state=42).fit(rf_daily[feature_cols_daily], rf_daily['total_revenue'])

# Forecast next 30 days recursively
future_dates_daily = pd.date_range(daily_sales.index[-1]+pd.Timedelta(days=1), periods=30)
future_preds_daily = []
last_values_daily = daily_sales['total_revenue'][-7:].tolist()
for d in future_dates_daily:
    features = [d.dayofyear, d.isocalendar().week, d.month, d.dayofweek] + last_values_daily[-7:]
    pred = rf_model_daily.predict([features])[0]
    future_preds_daily.append(pred)
    last_values_daily.append(pred)

rf_forecast_daily = pd.DataFrame({'total_revenue': future_preds_daily}, index=future_dates_daily)

# Add daily traces
fig.add_trace(go.Scatter(x=daily_sales.index, y=daily_sales['total_revenue'], name='Actual Daily', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=arima_forecast_daily.index, y=arima_forecast_daily['mean'], name='ARIMA Daily Forecast', line=dict(color='red', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=rf_forecast_daily.index, y=rf_forecast_daily['total_revenue'], name='RF Daily Forecast', line=dict(color='green', dash='dot')), row=1, col=1)

# -------------------- WEEKLY FORECASTING --------------------
weekly_sales = df.groupby(pd.Grouper(key='date', freq='W-MON'))['money'].sum().reset_index()
weekly_sales.columns = ['week', 'total_revenue']
weekly_sales.set_index('week', inplace=True)
weekly_sales = weekly_sales.asfreq('W-MON').fillna(method='ffill')

arima_weekly = ARIMA(weekly_sales['total_revenue'], order=(3,1,0)).fit()
arima_forecast_weekly = arima_weekly.get_forecast(steps=12).summary_frame()
arima_forecast_weekly['week'] = pd.date_range(weekly_sales.index[-1]+pd.Timedelta(weeks=1), periods=12, freq='W-MON')
arima_forecast_weekly.set_index('week', inplace=True)

rf_weekly = weekly_sales.copy()
rf_weekly['week_of_year'] = rf_weekly.index.isocalendar().week
rf_weekly['month'] = rf_weekly.index.month
rf_weekly['quarter'] = rf_weekly.index.quarter
for lag in range(1,5):
    rf_weekly[f'lag_{lag}'] = rf_weekly['total_revenue'].shift(lag)
rf_weekly.dropna(inplace=True)
feature_cols_weekly = ['week_of_year','month','quarter'] + [f'lag_{lag}' for lag in range(1,5)]
rf_model_weekly = RandomForestRegressor(n_estimators=100, random_state=42).fit(rf_weekly[feature_cols_weekly], rf_weekly['total_revenue'])

# Recursive 12-week forecast
future_weeks = pd.date_range(weekly_sales.index[-1]+pd.Timedelta(weeks=1), periods=12, freq='W-MON')
future_preds_weekly = []
last_values_weekly = weekly_sales['total_revenue'][-4:].tolist()
for w in future_weeks:
    features = [w.isocalendar().week, w.month, w.quarter] + last_values_weekly[-4:]
    pred = rf_model_weekly.predict([features])[0]
    future_preds_weekly.append(pred)
    last_values_weekly.append(pred)

rf_forecast_weekly = pd.DataFrame({'total_revenue': future_preds_weekly}, index=future_weeks)

# Add weekly traces
fig.add_trace(go.Scatter(x=weekly_sales.index, y=weekly_sales['total_revenue'], name='Actual Weekly', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=arima_forecast_weekly.index, y=arima_forecast_weekly['mean'], name='ARIMA Weekly Forecast', line=dict(color='red', dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=rf_forecast_weekly.index, y=rf_forecast_weekly['total_revenue'], name='RF Weekly Forecast', line=dict(color='green', dash='dot')), row=2, col=1)

# -------------------- MONTHLY FORECASTING --------------------
monthly_sales = df.groupby(pd.Grouper(key='date', freq='M'))['money'].sum().reset_index()
monthly_sales.columns = ['month','total_revenue']
monthly_sales.set_index('month', inplace=True)
monthly_sales = monthly_sales.asfreq('M').fillna(method='ffill')

arima_monthly = ARIMA(monthly_sales['total_revenue'], order=(2,1,0)).fit()
arima_forecast_monthly = arima_monthly.get_forecast(steps=6).summary_frame()
arima_forecast_monthly['month'] = pd.date_range(monthly_sales.index[-1]+pd.offsets.MonthEnd(1), periods=6, freq='M')
arima_forecast_monthly.set_index('month', inplace=True)

rf_monthly = monthly_sales.copy()
rf_monthly['month_of_year'] = rf_monthly.index.month
rf_monthly['quarter'] = rf_monthly.index.quarter
for lag in range(1,4):
    rf_monthly[f'lag_{lag}'] = rf_monthly['total_revenue'].shift(lag)
rf_monthly.dropna(inplace=True)
feature_cols_monthly = ['month_of_year','quarter'] + [f'lag_{lag}' for lag in range(1,4)]
rf_model_monthly = RandomForestRegressor(n_estimators=100, random_state=42).fit(rf_monthly[feature_cols_monthly], rf_monthly['total_revenue'])

# Recursive 6-month forecast
future_months = pd.date_range(monthly_sales.index[-1]+pd.offsets.MonthEnd(1), periods=6, freq='M')
future_preds_monthly = []
last_values_monthly = monthly_sales['total_revenue'][-3:].tolist()
for m in future_months:
    features = [m.month, m.quarter] + last_values_monthly[-3:]
    pred = rf_model_monthly.predict([features])[0]
    future_preds_monthly.append(pred)
    last_values_monthly.append(pred)

rf_forecast_monthly = pd.DataFrame({'total_revenue': future_preds_monthly}, index=future_months)

# Add monthly traces
fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales['total_revenue'], name='Actual Monthly', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=arima_forecast_monthly.index, y=arima_forecast_monthly['mean'], name='ARIMA Monthly Forecast', line=dict(color='red', dash='dash')), row=3, col=1)
fig.add_trace(go.Scatter(x=rf_forecast_monthly.index, y=rf_forecast_monthly['total_revenue'], name='RF Monthly Forecast', line=dict(color='green', dash='dot')), row=3, col=1)

# Layout
fig.update_layout(
    title='Comprehensive Sales Forecasting: Daily, Weekly, Monthly',
    height=1200,
    plot_bgcolor='white'
)
fig.show()

C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\3253306382.py:27: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoj

In [7]:
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ----------------- Data Preparation -----------------
# Ensure proper datetime
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df = df.dropna(subset=['datetime'])

# Floor timestamps to midnight to get 'date_only'
df['date_only'] = df['datetime'].dt.floor('D')
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.day_name()

# Group by date_only to sum sales
daily_sales = df.groupby('date_only')['money'].sum().reset_index()
daily_sales.columns = ['date', 'total_revenue']
daily_sales.set_index('date', inplace=True)

# Fill missing dates, but do not flatten variation
daily_sales = daily_sales.asfreq('D')

# ----------------- KPI Calculations -----------------
total_revenue = df['money'].sum()
total_transactions = len(df)
avg_transaction = df['money'].mean()
top_coffee = df.groupby("coffee_name")["money"].sum().idxmax()
top_customer = df.groupby("card")["money"].sum().idxmax()

# ----------------- Coffee Sales & Revenue -----------------
coffee_sales = df.groupby("coffee_name")["money"].sum().reset_index().sort_values("money", ascending=False)
fig_coffee_sales = px.bar(coffee_sales, x="coffee_name", y="money", title="Revenue by Coffee Type", color="money", color_continuous_scale="Tealgrn")
fig_revenue_contrib = px.treemap(coffee_sales, path=['coffee_name'], values='money', title='Revenue Contribution by Coffee Type')

# ----------------- Payment Behavior -----------------
payment_summary = df.groupby("cash_type")["money"].agg(['count','sum']).reset_index()
fig_payment_method = px.pie(payment_summary, names='cash_type', values='sum', title="Revenue by Payment Method")

coffee_payment = df.groupby(['coffee_name','cash_type']).size().reset_index(name='count')
coffee_payment['percent'] = coffee_payment.groupby('coffee_name')['count'].transform(lambda x: x / x.sum() * 100)
fig_coffee_payment_pct = px.bar(coffee_payment, x='coffee_name', y='percent', color='cash_type', barmode='stack', title='Payment Type Distribution per Coffee')

# ----------------- Hourly & Heatmap -----------------
hourly_sales = df.groupby('hour')['money'].sum().reset_index()
fig_hourly_sales = px.line(hourly_sales, x='hour', y='money', title='Sales by Hour', markers=True)

heatmap_data = df.groupby(['day_of_week','hour'])['money'].sum().reset_index()
heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='hour', values='money')
days_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
heatmap_pivot = heatmap_pivot.reindex(days_order)
fig_hourly_heatmap = px.imshow(heatmap_pivot, text_auto=True, aspect="auto", title='Sales Heatmap: Day vs Hour', color_continuous_scale='Teal')

# ----------------- Coffee Popularity Over Time (Weekly) -----------------
coffee_trend_weekly = df.groupby([pd.Grouper(key='datetime', freq='W'), 'coffee_name'])['money'].sum().reset_index()
fig_coffee_trend = px.line(coffee_trend_weekly, x='datetime', y='money', color='coffee_name', title='Coffee Popularity Over Time (Weekly)', line_shape='spline')

# ----------------- Forecasting: Daily, Weekly, Monthly -----------------
from plotly.subplots import make_subplots
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
import pandas as pd

# Ensure datetime
df['date'] = pd.to_datetime(df['date_only'])

# -------------------- DAILY FORECAST FIGURE --------------------
daily_sales = df.groupby('date')['money'].sum().reset_index()
daily_sales.columns = ['date', 'total_revenue']
daily_sales.set_index('date', inplace=True)
daily_sales = daily_sales.asfreq('D').fillna(method='ffill')

# ARIMA forecast
arima_daily = ARIMA(daily_sales['total_revenue'], order=(5,1,0)).fit()
arima_forecast_daily = arima_daily.get_forecast(steps=30).summary_frame()
arima_forecast_daily['date'] = pd.date_range(daily_sales.index[-1] + pd.Timedelta(days=1), periods=30)
arima_forecast_daily.set_index('date', inplace=True)

# RF forecast
rf_daily = daily_sales.copy()
rf_daily['day_of_year'] = rf_daily.index.dayofyear
rf_daily['week_of_year'] = rf_daily.index.isocalendar().week
rf_daily['month'] = rf_daily.index.month
rf_daily['day_of_week'] = rf_daily.index.dayofweek
for lag in range(1, 8):
    rf_daily[f'lag_{lag}'] = rf_daily['total_revenue'].shift(lag)
rf_daily.dropna(inplace=True)
feature_cols_daily = ['day_of_year','week_of_year','month','day_of_week'] + [f'lag_{lag}' for lag in range(1,8)]
rf_model_daily = RandomForestRegressor(n_estimators=100, random_state=42).fit(rf_daily[feature_cols_daily], rf_daily['total_revenue'])

future_dates_daily = pd.date_range(daily_sales.index[-1]+pd.Timedelta(days=1), periods=30)
future_preds_daily = []
last_values_daily = daily_sales['total_revenue'][-7:].tolist()
for d in future_dates_daily:
    features = [d.dayofyear, d.isocalendar().week, d.month, d.dayofweek] + last_values_daily[-7:]
    pred = rf_model_daily.predict([features])[0]
    future_preds_daily.append(pred)
    last_values_daily.append(pred)
rf_forecast_daily = pd.DataFrame({'total_revenue': future_preds_daily}, index=future_dates_daily)

# Daily figure
fig_forecast_daily = go.Figure()
fig_forecast_daily.add_trace(go.Scatter(x=daily_sales.index, y=daily_sales['total_revenue'], name='Actual Daily', line=dict(color='blue')))
fig_forecast_daily.add_trace(go.Scatter(x=arima_forecast_daily.index, y=arima_forecast_daily['mean'], name='ARIMA Daily Forecast', line=dict(color='red', dash='dash')))
fig_forecast_daily.add_trace(go.Scatter(x=rf_forecast_daily.index, y=rf_forecast_daily['total_revenue'], name='RF Daily Forecast', line=dict(color='green', dash='dot')))


# -------------------- WEEKLY FORECAST FIGURE --------------------
weekly_sales = df.groupby(pd.Grouper(key='date', freq='W-MON'))['money'].sum().reset_index()
weekly_sales.columns = ['week', 'total_revenue']
weekly_sales.set_index('week', inplace=True)
weekly_sales = weekly_sales.asfreq('W-MON').fillna(method='ffill')

arima_weekly = ARIMA(weekly_sales['total_revenue'], order=(3,1,0)).fit()
arima_forecast_weekly = arima_weekly.get_forecast(steps=12).summary_frame()
arima_forecast_weekly['week'] = pd.date_range(weekly_sales.index[-1]+pd.Timedelta(weeks=1), periods=12, freq='W-MON')
arima_forecast_weekly.set_index('week', inplace=True)

rf_weekly = weekly_sales.copy()
rf_weekly['week_of_year'] = rf_weekly.index.isocalendar().week
rf_weekly['month'] = rf_weekly.index.month
rf_weekly['quarter'] = rf_weekly.index.quarter
for lag in range(1,5):
    rf_weekly[f'lag_{lag}'] = rf_weekly['total_revenue'].shift(lag)
rf_weekly.dropna(inplace=True)
feature_cols_weekly = ['week_of_year','month','quarter'] + [f'lag_{lag}' for lag in range(1,5)]
rf_model_weekly = RandomForestRegressor(n_estimators=100, random_state=42).fit(rf_weekly[feature_cols_weekly], rf_weekly['total_revenue'])

future_weeks = pd.date_range(weekly_sales.index[-1]+pd.Timedelta(weeks=1), periods=12, freq='W-MON')
future_preds_weekly = []
last_values_weekly = weekly_sales['total_revenue'][-4:].tolist()
for w in future_weeks:
    features = [w.isocalendar().week, w.month, w.quarter] + last_values_weekly[-4:]
    pred = rf_model_weekly.predict([features])[0]
    future_preds_weekly.append(pred)
    last_values_weekly.append(pred)
rf_forecast_weekly = pd.DataFrame({'total_revenue': future_preds_weekly}, index=future_weeks)

# Weekly figure
fig_forecast_weekly = go.Figure()
fig_forecast_weekly.add_trace(go.Scatter(x=weekly_sales.index, y=weekly_sales['total_revenue'], name='Actual Weekly', line=dict(color='blue')))
fig_forecast_weekly.add_trace(go.Scatter(x=arima_forecast_weekly.index, y=arima_forecast_weekly['mean'], name='ARIMA Weekly Forecast', line=dict(color='red', dash='dash')))
fig_forecast_weekly.add_trace(go.Scatter(x=rf_forecast_weekly.index, y=rf_forecast_weekly['total_revenue'], name='RF Weekly Forecast', line=dict(color='green', dash='dot')))


# -------------------- MONTHLY FORECAST FIGURE --------------------
monthly_sales = df.groupby(pd.Grouper(key='date', freq='M'))['money'].sum().reset_index()
monthly_sales.columns = ['month','total_revenue']
monthly_sales.set_index('month', inplace=True)
monthly_sales = monthly_sales.asfreq('M').fillna(method='ffill')

arima_monthly = ARIMA(monthly_sales['total_revenue'], order=(2,1,0)).fit()
arima_forecast_monthly = arima_monthly.get_forecast(steps=6).summary_frame()
arima_forecast_monthly['month'] = pd.date_range(monthly_sales.index[-1]+pd.offsets.MonthEnd(1), periods=6, freq='M')
arima_forecast_monthly.set_index('month', inplace=True)

rf_monthly = monthly_sales.copy()
rf_monthly['month_of_year'] = rf_monthly.index.month
rf_monthly['quarter'] = rf_monthly.index.quarter
for lag in range(1,4):
    rf_monthly[f'lag_{lag}'] = rf_monthly['total_revenue'].shift(lag)
rf_monthly.dropna(inplace=True)
feature_cols_monthly = ['month_of_year','quarter'] + [f'lag_{lag}' for lag in range(1,4)]
rf_model_monthly = RandomForestRegressor(n_estimators=100, random_state=42).fit(rf_monthly[feature_cols_monthly], rf_monthly['total_revenue'])

future_months = pd.date_range(monthly_sales.index[-1]+pd.offsets.MonthEnd(1), periods=6, freq='M')
future_preds_monthly = []
last_values_monthly = monthly_sales['total_revenue'][-3:].tolist()
for m in future_months:
    features = [m.month, m.quarter] + last_values_monthly[-3:]
    pred = rf_model_monthly.predict([features])[0]
    future_preds_monthly.append(pred)
    last_values_monthly.append(pred)
rf_forecast_monthly = pd.DataFrame({'total_revenue': future_preds_monthly}, index=future_months)

# Monthly figure
fig_forecast_monthly = go.Figure()
fig_forecast_monthly.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales['total_revenue'], name='Actual Monthly', line=dict(color='blue')))
fig_forecast_monthly.add_trace(go.Scatter(x=arima_forecast_monthly.index, y=arima_forecast_monthly['mean'], name='ARIMA Monthly Forecast', line=dict(color='red', dash='dash')))
fig_forecast_monthly.add_trace(go.Scatter(x=rf_forecast_monthly.index, y=rf_forecast_monthly['total_revenue'], name='RF Monthly Forecast', line=dict(color='green', dash='dot')))

# ----------------- Cumulative Sales -----------------
cumulative_sales = daily_sales.cumsum().reset_index()
fig_cumulative = px.line(cumulative_sales, x='date', y='total_revenue', title='Cumulative Sales Over Time')

# ----------------- Dash App -----------------
app = dash.Dash(__name__)
app.layout = html.Div(style={'padding':'20px','font-family':'Arial, sans-serif'}, children=[
    html.H1("Advanced Coffee Sales Dashboard", style={'textAlign':'center','color':'#0a3d62'}),

    # KPI Row
    html.Div([
        html.Div([html.H4("Total Revenue"), html.P(f"${total_revenue:,.2f}")],
                 style={'width':'23%','display':'inline-block','textAlign':'center','background':'#dff9fb','padding':'15px','margin':'5px','border-radius':'10px'}),
        html.Div([html.H4("Total Transactions"), html.P(f"{total_transactions:,}")],
                 style={'width':'23%','display':'inline-block','textAlign':'center','background':'#dff9fb','padding':'15px','margin':'5px','border-radius':'10px'}),
        html.Div([html.H4("Avg Transaction"), html.P(f"${avg_transaction:,.2f}")],
                 style={'width':'23%','display':'inline-block','textAlign':'center','background':'#dff9fb','padding':'15px','margin':'5px','border-radius':'10px'}),
        html.Div([html.H4("Top Coffee"), html.P(f"{top_coffee}")],
                 style={'width':'23%','display':'inline-block','textAlign':'center','background':'#dff9fb','padding':'15px','margin':'5px','border-radius':'10px'}),
    ], style={'display':'flex','justifyContent':'space-around','margin-bottom':'20px'}),

    # Row 1: Coffee & Payment
    html.Div([
        html.Div([dcc.Graph(figure=fig_coffee_sales)], style={'width':'48%','display':'inline-block'}),
        html.Div([dcc.Graph(figure=fig_payment_method)], style={'width':'48%','display':'inline-block'}),
    ], style={'display':'flex','justifyContent':'space-between','margin-bottom':'20px'}),

    # Row 2: Coffee Payment %
    html.Div([
        html.Div([dcc.Graph(figure=fig_coffee_payment_pct)], style={'width':'48%','display':'inline-block'}),
        html.Div([dcc.Graph(figure=fig_cumulative)], style={'width':'48%','display':'inline-block'}),
    ], style={'display':'flex','justifyContent':'space-between','margin-bottom':'20px'}),

    # Row 3: Hourly & Heatmap
    html.Div([
        html.Div([dcc.Graph(figure=fig_hourly_sales)], style={'width':'48%','display':'inline-block'}),
        html.Div([dcc.Graph(figure=fig_hourly_heatmap)], style={'width':'48%','display':'inline-block'}),
    ], style={'display':'flex','justifyContent':'space-between','margin-bottom':'20px'}),

    # Row 4: Coffee trends
    html.Div([dcc.Graph(figure=fig_coffee_trend)], style={'margin-bottom':'20px'}),

    # Row 5: Forecasts (Tabs)
    html.Div([
        html.H3("Sales Forecasts"),
        dcc.Tabs([
            dcc.Tab(label='Daily', children=[dcc.Graph(figure=fig_forecast_daily)]),
            dcc.Tab(label='Weekly', children=[dcc.Graph(figure=fig_forecast_weekly)]),
            dcc.Tab(label='Monthly', children=[dcc.Graph(figure=fig_forecast_monthly)])
        ])
    ])
])

if __name__ == "__main__":
    app.run(debug=True)


C:\Users\shrey\AppData\Local\Temp\ipykernel_9856\1189734608.py:77: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

g:\MCA Practice\MLporoj